In [34]:
pip install pandas

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached pandas-2.2.1-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


You should consider upgrading via the 'c:\Users\A507\sstrinic\lab3\labs\Scripts\python.exe -m pip install --upgrade pip' command.


In [55]:
import re
import datetime
import pandas as pd

log_path="setupapi.dev2.log"
regex = "r'^>>>  \[Device Install.*#(Disk&Ven_[A-Za-z0-9]+)&(Prod_([\w\s\S]+?))&(Rev_([\w\s\S]+?))#([\w\s\S]+?)#.*\]'"
usb_devices = []

# Read the contents of the setupapi.dev.log file
with open(log_path, "r") as log_file:
     # Store information about each USB device in a dictionary
     for line in log_file:
        match=re.match(r'^>>>  \[Device Install.*#(Disk&Ven_[A-Za-z0-9]+)&(Prod_([\w\s\S]+?))&(Rev_([\w\s\S]+?))#([\w\s\S]+?)#.*\]',line)
        
        if(not match):
            continue
        device_vendor_id=match.group(1)
        device_product_id=match.group(2)
        device_instance_id=match.group(4)
        serial_number=match.group(6)
        usb_devices.append({
            "VENDOR ID": device_vendor_id,
            "PRODUCT ID": device_product_id,
            "INSTANCE ID": device_instance_id,
            "SERIAL No.": serial_number,
            #"TIMESTAMP": re.findall(r'\b\d{4}/\d{2}/\d{2}\s\d{2}\.:\d{2}:\d{2}\.\d{3}\b', next(log_file))[0]
            "TIMESTAMP": next(log_file).split("start")[1].split("\n")[0]
        })
        

# create a Pandas dataframe with the file names
df = pd.DataFrame(usb_devices)

# print the dataframe
df.tail(20)

,VENDOR ID,PRODUCT ID,INSTANCE ID,SERIAL No.,TIMESTAMP
0,Disk&Ven_Verbatim,Prod_STORE_N_GO,Rev_1100,1311130000000480&0,2022/11/03 15:00:17.389
1,Disk&Ven_SMI,Prod_USB_DISK,Rev_1100,6&7d7590d&0,2023/02/01 11:10:46.945
2,Disk&Ven_VendorCo,Prod_ProductCode,Rev_2.00,920707402A3F8066956&0,2023/02/06 12:50:42.455
3,Disk&Ven_VendorCo,Prod_ProductCode,Rev_2.00,9207113E5CF83172266&0,2023/02/06 12:51:00.660
4,Disk&Ven_Kingston,Prod_DataTraveler_2.0,Rev_PMAP,AC220B280549EF6059EEBAFE&0,2023/02/06 12:52:25.253
5,Disk&Ven_Kingston,Prod_DataTraveler_2.0,Rev_PMAP,AC220B280549EF6059EEBAFE&0,2023/03/23 08:13:57.618
6,Disk&Ven_JetFlash,Prod_TS128MJF2A,Rev_1.00,6&22ff249f&1,2023/03/29 16:28:31.704
7,Disk&Ven_JetFlash,Prod_TS128MJF2A,Rev_1.00,6&104573fa&1,2023/03/29 16:32:50.762


In [54]:
import os
import pandas as pd
import requests
import hashlib
import magic
import mimetypes
import time

# specify the directory path where the files are located
dir_path = 'E:\\'

# create an empty list to store the file names
file_names = []
extensions = []
md5s = []
sha1s = []
sha256s = []
magic_numbers = []
fa = magic.Magic(mime=True)
original_extensions = []
extension_matches = []
creation_times=[]
modfication_times=[]
access_times=[]

def file2hash(filename, hash_function):
    with open(filename, 'rb', buffering=0) as f:
        return hash_function(f.read()).hexdigest()

# iterate through all files in the directory
for file in os.listdir(dir_path):
    path = os.path.join(dir_path, file)
    # check if the file is a regular file (i.e., not a directory)
    if os.path.isfile(os.path.join(dir_path, file)):
        name,extension=os.path.splitext(file)
        file_names.append(file)
        extensions.append(extension)
        
        md5s.append(file2hash(path, hashlib.md5))
        sha1s.append(file2hash(path, hashlib.sha1))
        sha256s.append(file2hash(path, hashlib.sha256))
        
        creation_times.append(time.ctime(os.path.getctime(os.path.join(dir_path, file))))
        modfication_times.append(time.ctime(os.path.getmtime(os.path.join(dir_path, file))))
        access_times.append(time.ctime(os.path.getatime(os.path.join(dir_path, file))))
        

        # magic_num=fa.from_file(path)
        # magic_numbers.append(magic_num)
        # original_extensions.append(mimetypes.guess_all_extensions(magic_num.lower())[0])



# create a Pandas dataframe with the file names
df = pd.DataFrame({'file_name': file_names, 'extension': extensions, 'md5': md5s, 'sha1': sha1s, 'sha256': sha256s, 'creation_time': creation_times, 'modification_times': modfication_times,'access_times': access_times})

# print the dataframe
df.tail(20)

,file_name,extension,md5,sha1,sha256,creation_time,modification_times,access_times
0,Secret_file_11.txt,.txt,187b0341b64a0e16d74ea3b50e22e6c2,08152ff79a16f2a181174eb5fe8de0197f4d722f,5f90ecf090772e3366353171a2e8b9c4001934cdc3a5e0...,Wed Mar 29 17:35:08 2023,Mon Oct 28 16:08:46 2019,Wed Mar 29 01:00:00 2023
1,Secret_file_92.pdf,.pdf,4266ea20320c598bc5d7b1d731e3e9e9,1a30a1d139131e7105a1896d7719c85c03003a59,3e174794698cbdf84d9a5cd352f755d0e6bdfc5dc65e0d...,Wed Mar 29 17:35:10 2023,Mon Oct 28 16:09:32 2019,Wed Mar 29 01:00:00 2023


In [ ]:
JetFlash je koristen Prod_TS128MJF2A.